In [1]:
from numpy.linalg import eig
import numpy as np
import cv2
import os
import sys
import pickle

In [2]:
configuration_file = 'config/part_1.cfg'

**Main: process_video.py**

In [6]:
from src.extract_features import *
from src.parsing import *
from src.display_video import *
from main import *

config_data = parse_configuration_file(configuration_file)     #sys.argv[1]) #Parse the configuration file
match_img1 , match_map = parse_points(config_data) #Parse the points from the configuration file
video_path, type_homography, file_name_keypoints, file_name_tranformations = final_parsing(config_data)

sift_points, nr_points = extract_features(video_path)


image matches:  [('225', '131'), ('580', '120'), ('626', '305'), ('133', '303')]
map matches:  [('225', '131'), ('580', '120'), ('626', '305'), ('133', '303')] 

Total frames of the video:  1901
(Nº features, Nº descriptors per feature):  (5000, 128)
Nº of frames extracted:  20


Output of process_video.py:

In [7]:
from src.outputs import *
create_output_keypoints(sift_points, file_name_keypoints, nr_points) #teste


**Main: compute_transform.py**

In [8]:
from src.matching_features import *
from src.homography import *
from src.ransac import *
from src.view_homography import *
from main import *

H_frame1_to_map =compute_homography(match_img1, match_map)    
match = matching_features_SCIKITLEARN(sift_points)
H_sequential = create_sequential_homographies(match, sift_points)
if type_homography =='map':
    H_output = homography_to_map(H_sequential, H_frame1_to_map)
elif type_homography =='all':
    H_output = all_homographies(H_sequential)



Output of compute_transform.py:

In [9]:
from src.outputs import *
create_output(H_output, file_name_tranformations)        


Saída salva em: outputs/drone_transforms.mat


**Display results:**

In [10]:
def show_pixel_coordinates(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Clicked at (x={x}, y={y})")

def display_frame(name,frame):
    cv2.namedWindow(name)
    cv2.setMouseCallback(name, show_pixel_coordinates)
    cv2.imshow(name, frame)

def display(frame1, frame2,homography_de2_para1 , idx1, idx2 ):
    H= homography_de2_para1.reshape((3,3))
    
    height, width = frame2.shape[:2]
    # Apply homography to frame 2
    warped_frame2 = cv2.warpPerspective(frame2, H, (width, height))

    while True:
        display_frame(f'Frame {idx1}', frame1)

        display_frame(f'Frame{ idx2} transformed with homography to coordinates of frame {idx1}', warped_frame2)

        display_frame(f"Frame {idx2}", frame2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        elif cv2.waitKey(1) & 0xFF == ord('e'):
            return True
    cv2.destroyAllWindows()
    return False

def extract__frames(video_path):
    """Extracts the features from the video and stores them in a list"""
    print(video_path)
    capture = cv2.VideoCapture(os.path.abspath(video_path))
    k = 0
    frames=[]
    count_frames(video_path)
    while k <= 1900:
        capture.set(cv2.CAP_PROP_POS_FRAMES, k)
        success, frame = capture.read() #read the video
        if success:
            frames.append(frame)
        k += 100
    return frames

In [11]:
frames= extract__frames('video/trymefirst_lisbon.mp4')

video/trymefirst_lisbon.mp4
Total frames of the video:  1901


In [12]:
#to test homography to map
img1=frames[0]
idx1 = 0
print('Press q to change image\nPress e to exit')

for i in range(1,len(frames)-1):
    idx2 = i 
    img2= frames[i]
    exit_bool = display(img1, img2, H_output[2:,i], idx1, idx2 )
    if exit_bool:
        cv2.destroyAllWindows()
        break

Press q to change image
Press e to exit
Clicked at (x=561, y=127)
Clicked at (x=255, y=117)


In [13]:
#to test homography to each other
print('Press q to change image\nPress e to exit')
for i in range(H_output.shape[1]):
    idx1= int(H_output[0,i])
    idx2= int(H_output[1,i])

    img1 = frames[ idx1-1 ] 
    img2= frames[  idx2-1]
    exit_bool = display(img1, img2, H_output[2:,i], idx1, idx2 )
    if exit_bool:
        cv2.destroyAllWindows()
        break

Press q to change image
Press e to exit
Clicked at (x=390, y=185)
Clicked at (x=271, y=148)
Clicked at (x=271, y=148)
Clicked at (x=329, y=191)
Clicked at (x=280, y=170)
Clicked at (x=280, y=170)
Clicked at (x=165, y=218)


KeyboardInterrupt: 